In [1]:
#import numpy as np
#from numpy import dot
#from numpy.linalg import norm

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tracers import ConsoleCallbackHandler
from langchain_community.chat_message_histories import ChatMessageHistory

import os
from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("Doogie.2ndKey")
os.environ['OPENAI_API_KEY'] = api_key

myllm = ChatOpenAI(
    temperature=0.1,
    max_tokens=2048,
    model='gpt-3.5-turbo',
)

question = "세종대왕이 누구인지 설명해줘"

result = myllm.invoke( question )
print(result.content)
#print(result)

세종대왕(1397년~1450년)은 조선시대 4대 왕 중 한 명으로, 조선시대의 제4대 군주이자 문신이기도 했습니다. 그는 조선시대를 대표하는 왕으로, 한글을 창제하고 문화 발전에 큰 기여를 한 인물로 알려져 있습니다. 세종대왕은 국정을 잘 다스리고 국민들의 복지를 증진시키는 데 많은 노력을 기울였으며, 또한 과학기술과 문화예술을 발전시키는 데도 큰 관심을 가졌습니다. 그의 통치는 조선시대의 전성기를 이루는 데 큰 역할을 했으며, 그의 이념과 정치적 업적은 한국 역사상 큰 영향을 끼쳤습니다. 현재 한글날은 그의 생일인 10월 9일을 기념하는 날로 지정되어 있습니다.
content='세종대왕(1397년~1450년)은 조선시대 4대 왕 중 한 명으로, 조선시대의 제4대 군주이자 문신이기도 했습니다. 그는 조선시대를 대표하는 왕으로, 한글을 창제하고 문화 발전에 큰 기여를 한 인물로 알려져 있습니다. 세종대왕은 국정을 잘 다스리고 국민들의 복지를 증진시키는 데 많은 노력을 기울였으며, 또한 과학기술과 문화예술을 발전시키는 데도 큰 관심을 가졌습니다. 그의 통치는 조선시대의 전성기를 이루는 데 큰 역할을 했으며, 그의 이념과 정치적 업적은 한국 역사상 큰 영향을 끼쳤습니다. 현재 한글날은 그의 생일인 10월 9일을 기념하는 날로 지정되어 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 312, 'prompt_tokens': 26, 'total_tokens': 338, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_

In [2]:
from langchain_core.prompts import PromptTemplate

mytemplate = "{who}가 누구인지 설명해 줘."

myprompt = PromptTemplate(
    template=mytemplate, input_variables={'who'}
)

print(myprompt)
print(myprompt.format(who="오바마"))

input_variables=['who'] input_types={} partial_variables={} template='{who}가 누구인지 설명해 줘.'
오바마가 누구인지 설명해 줘.


In [3]:
myllm_chain = myprompt | myllm
result = myllm_chain.invoke( {"who:이순신장군"} )
print(result.content)

이순신 장군은 조선 시대의 무신이자 민족영웅으로, 조선 중기에 살았던 유명한 무장이다. 그는 일제강점기에 신라와의 전쟁에서 승리를 거두며 일본의 침략을 막는 데 큰 공헌을 했다. 특히, 한산도 대첩에서 대대적인 승리를 거둬 일본의 해군을 물리쳤던 이순신 장군은 조선사 대표적인 무장 중 한 명으로 인정받고 있다.


In [8]:
from langchain_core.tracers import ConsoleCallbackHandler

result = myllm_chain.invoke(
    {"who:강감찬장군"},
    config={"callback": [ConsoleCallbackHandler()] },
)
print(result.content)

강감찬 장군은 고려시대에 활약한 무신으로, 고려의 대표적인 무장이자 무신 중 한 명입니다. 그는 안창호와 함께 고려의 동북방 국경을 지키며 여러 차례 위기를 극복하고 승리를 거두었습니다. 특히 안동성 전투와 천안성 전투에서 대승을 거둬 고려의 위기를 막는 데 큰 기여를 했습니다. 그의 뛰어난 전략과 무모한 전투 스타일은 후세에도 전투 전략가들에게 많은 영향을 주었습니다. 현재까지도 그의 업적은 많은 사람들에게 존경받고 있습니다.


In [26]:
from langchain_community.chat_message_histories import ChatMessageHistory

mytemplate = """
    아래는 사람과 AI의 친근한 대화입니다. AI의 이름은 마이봇 입니다.
    대화 문맥을 바탕으로 친정한 답변을 진행해라

    Current Conversation:
    {myhistory}

    Human: {myinput}
    AI:
"""
myprompt = PromptTemplate(
    template=mytemplate, input_variables={'myshitory', 'myinput'}
)

myllm = ChatOpenAI( model='gpt-3.5-turbo',)
mychain = myprompt | myllm

mystore = {}
session_id = "myFirstSession"

if session_id not in mystore:
    mystore[session_id] = ChatMessageHistory()
mysessionhistory = mystore[session_id]

mymsghistory = RunnableWithMessageHistory(
    mychain,
    lambda session_id: mysessionhistory,
    input_messages_key="myinput",
    history_messages_key="myhistory"
)

result = mymsghistory.invoke(
    {"myinput": "너는 어디에서 만들었는가?"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print(result.content)

result = mymsghistory.invoke(
    {"myinput": "너의 이름은 무엇인가?"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print(result.content)

마이봇: 나는 온라인에서 만들어졌어! 인터넷 세계의 한 곳에 살고 있지~
나는 마이봇이라고 해! 너와 함께 대화하는 것을 즐기고 있어.


In [28]:
result = mymsghistory.invoke(
    {"myinput": "푸른 바다를 주제로 해학과 중의가 넘치는 짧은 시를 만들어줘"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print(result.content)

너무 아름다운 푸른 바다
물결이 춤을 추며 흐른다
바람과 함께 자유롭게 춤을 추는
바다 속의 물고기들의 행복한 노래


In [30]:
result = mymsghistory.invoke(
    {"myinput": "석양을 주제로도 해줘"},
    config={"configurable": {"session_id": "myFirstSession"} },
)
print(result.content)

석양의 황금빛이 바다 위로 노래를 부르며 살랑살랑 흔든다. 저기 저 바다를 향해 선배들이 돌아가고 있어. 종소리와 함께 하루가 저물고, 저 멀리 솟아오르는 달이 조용히 반짝이고 있어.


In [39]:
print("mystore:", len(mystore.keys() ) )

mystore: 1


In [ ]:
print("mytemplate:", mytemplate)
print("myprompt:", myprompt)
print("myllm:", myllm)
print("mychain:", mychain)
print("mystore:", mystore)
print("session_id:", session_id)
print("mystore[",session_id,"]=", mystore )
print("mysessionhistory:", mysessionhistory)
